In [1]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd

In [2]:
laps = pd.read_pickle('clean/96/laps').dropna()
laps['Track'] = pd.to_numeric(laps.index.get_level_values(0))
laps = laps[laps['Events'] == '-'][['Lap time', 'Tyres', 'Weather', 'Temp', 'Hum', 'Track']]
laps['Tyres'] = laps['Tyres'].str.split('(').str[0] # Worn tyres append a (W), remove that
laps

0     Lap time Tyres      Weather  Temp   Hum  Track
12 2    88.769  Soft  Very Cloudy    31  0.46     12
   3    79.039  Soft  Very Cloudy    31  0.46     12
   4      79.7  Soft  Very Cloudy    31  0.46     12
   5    79.088  Soft  Very Cloudy    31  0.46     12
   6    79.425  Soft  Very Cloudy    31  0.46     12
...        ...   ...          ...   ...   ...    ...
11 54   98.968  Rain         Rain    32  0.86     11
   56  134.593  Rain         Rain    31  0.86     11
   57  101.646  Rain         Rain    31  0.86     11
   58  100.089  Rain         Rain    31  0.86     11
   59  100.303  Rain         Rain    31  0.86     11

[352 rows x 6 columns]

In [3]:
tracks = pd.read_pickle('clean/96/tracks')
tracks

,Track name,Country,Kms,Laps,Lap distance,Power,Handling,Acceleration,Category,GPs held
1,Interlagos,Brazil,305.9,71,4.308,NaN,NaN,NaN,F1,33
2,Melbourne,Australia,307.6,58,5.303,NaN,NaN,NaN,F1,33
3,Mexico City,Mexico,305.0,69,4.420,NaN,NaN,NaN,F1,41
4,Buenos Aires,Argentina,306.6,72,4.258,NaN,NaN,NaN,ex F1,35
5,Las Vegas,United States,305.9,50,6.118,NaN,NaN,NaN,F1,1
6,Fuji,Japan,305.4,67,4.558,NaN,NaN,NaN,ex F1,36
7,Yas Marina,United Arab Emirates,305.5,55,5.555,NaN,NaN,NaN,F1,39
8,Shanghai,China,305.2,56,5.450,NaN,NaN,NaN,F1,36
9,Istanbul,Turkey,309.4,58,5.334,NaN,NaN,NaN,ex F1,36
10,Sepang,Malaysia,310.4,55,5.644,NaN,NaN,NaN,ex F1,39


In [4]:
laps['Lap distance'] = tracks.loc[laps['Track'], 'Lap distance'].values
laps['Laps'] = tracks.loc[laps['Track'], 'Laps'].values
laps.drop(columns='Track', inplace=True)
laps

0     Lap time Tyres      Weather  Temp   Hum  Lap distance  Laps
12 2    88.769  Soft  Very Cloudy    31  0.46         4.573    67
   3    79.039  Soft  Very Cloudy    31  0.46         4.573    67
   4      79.7  Soft  Very Cloudy    31  0.46         4.573    67
   5    79.088  Soft  Very Cloudy    31  0.46         4.573    67
   6    79.425  Soft  Very Cloudy    31  0.46         4.573    67
...        ...   ...          ...   ...   ...           ...   ...
11 54   98.968  Rain         Rain    32  0.86         5.247    58
   56  134.593  Rain         Rain    31  0.86         5.247    58
   57  101.646  Rain         Rain    31  0.86         5.247    58
   58  100.089  Rain         Rain    31  0.86         5.247    58
   59  100.303  Rain         Rain    31  0.86         5.247    58

[352 rows x 7 columns]

In [5]:
col_trans = ColumnTransformer([
    ('tyres', OneHotEncoder(), ['Tyres']),
    ('weather', OneHotEncoder(), ['Weather']),
], remainder='passthrough')

Adaboost

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [7]:
X = col_trans.fit_transform(laps.drop(columns=['Lap time']))
y = laps['Lap time']

In [8]:
ada = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 4), n_estimators = 100)
cross_val_score(ada, X, y, cv = 20).mean()

-10.618647160602743

In [9]:
ada.fit(X, y)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=4),
                  n_estimators=100)

In [10]:
params = {'Weather': 'Cloudy', 'Temp': 30, 'Hum': 0.75, 'Lap distance': 4.692, 'Laps': 66}
tyres = ['Extra Soft', 'Soft', 'Medium', 'Hard', 'Rain']

In [11]:
new = pd.DataFrame(5*[params])
new['Tyres'] = tyres
new

,Weather,Temp,Hum,Lap distance,Laps,Tyres
0,Cloudy,30,0.75,4.692,66,Extra Soft
1,Cloudy,30,0.75,4.692,66,Soft
2,Cloudy,30,0.75,4.692,66,Medium
3,Cloudy,30,0.75,4.692,66,Hard
4,Cloudy,30,0.75,4.692,66,Rain


In [12]:
ada.predict(col_trans.transform(new))

array([ 95.136     , 100.42471795, 100.42471795, 100.42471795,
       100.42471795])